In [8]:
import numpy as np
import analysis
s = np.array([0, 0, 1, 0, 2, 2, 2])
n = 3
m = 4
seq= np.array([[0, 1, 1, 1],
               [0, 0, 1, 0],
               [0, 0, 1, 0],
               [0, 1, 0, 0],
               [1, 0, 0, 1],
               [1, 1, 0, 0],
               [0, 1, 0, 1]])

hmm = analysis.HMM4binary_sensors()
hmm.fit_with_true_hidden_states(seq, s)
print(hmm.predict_states(seq))
print(hmm.A)
print(hmm.P)

(array([0, 1, 0, 2, 2, 2, 2]), 5.873606637142921e-07)
[[0.33333333 0.33333333 0.33333333]
 [1.         0.         0.        ]
 [0.         0.         1.        ]]
[[0.         0.66666667 0.66666667 0.33333333]
 [0.         0.         1.         0.        ]
 [0.66666667 0.66666667 0.         0.66666667]]


In [3]:
import numpy as np
import analysis
C = np.array([1/2, 1/2])
A = np.array([[1/4, 3/4],
              [1/4, 3/4]])
P = np.array([[4/5, 1/5],
              [1/5, 2/3]])
s = np.random.randint(0, 2)
states = [s]
data = [[np.random.binomial(1, P[s][0]), np.random.binomial(1, P[s][1])]]
for i in range(10000):
    flag = np.random.binomial(1, A[s][0])
    if s == 0:        
        if not(flag):
            s = 1
    else:
        if flag:
            s = 0
    states.append(s)
    data.append([np.random.binomial(1, P[s][0]), np.random.binomial(1, P[s][1])])
states = np.array(states)
data = np.array(data)

hmm = analysis.HMM4binary_sensors()
hmm.fit_with_true_hidden_states(data, states)
print(hmm.C)
print(hmm.A)
print(hmm.P)

[0. 1.]
[[0.25408204 0.74591796]
 [0.25023368 0.74976632]]
[[0.79299363 0.21218153]
 [0.20269729 0.67071705]]


In [13]:
a = np.array([True, False, True, False])
b = np.array([False, False, True, True])
a*b

array([False, False,  True, False])

In [16]:
a = np.array([[1, 0],
              [0, 1],
               [1, 0]])
a == a[0]

array([[ True,  True],
       [False, False],
       [ True,  True]])